In [1]:

import pandas as pd
import re
import os
from google.colab import drive

In [ ]:

drive.mount('/content/drive')

In [3]:
input_folder_path  = '/content/drive/MyDrive/SustainabilityReports/Firm_ID/Results/Merged'

In [4]:
lookup_file_path = '/content/drive/MyDrive/SustainabilityReports/Firm_ID/Results/DB/esg_indicator_aliases.csv'

In [5]:
company_id_lookup_file_path ='/content/drive/MyDrive/SustainabilityReports/Firm_ID/Results/DB/company_id_alias.csv'

In [6]:
quantitative_output_file_path = '/content/drive/MyDrive/SustainabilityReports/Firm_ID/Results/DB/quantitative_counts.csv'
indicator_output_file_path = '/content/drive/MyDrive/SustainabilityReports/Firm_ID/Results/DB/indicator_counts.csv'


In [7]:
lookup_df = pd.read_csv(lookup_file_path)
company_id_lookup_df = pd.read_csv(company_id_lookup_file_path, encoding='ISO-8859-1')


In [ ]:
lookup_df.columns = lookup_df.columns.str.strip()
company_id_lookup_df.columns = company_id_lookup_df.columns.str.strip()

lookup_df.rename(columns={'indicator': 'Indicator'}, inplace=True)
company_id_lookup_df.rename(columns={'company_alias': 'Company'}, inplace=True)

# Create lookup dictionaries
lookup_dict = pd.Series(lookup_df['Alias'].values, index=lookup_df['Indicator'].str.lower()).to_dict()
company_id_lookup_dict = pd.Series(company_id_lookup_df['company_id'].values, index=company_id_lookup_df['Company'].str.lower()).to_dict()

#  company IDs with partial matching
def match_company_id(company_name, lookup_dict):
    for alias, company_id in lookup_dict.items():
        if alias in company_name.lower():
            return company_id
    return None


all_quantitative_counts = pd.DataFrame()
all_indicator_counts = pd.DataFrame()

for filename in os.listdir(input_folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(input_folder_path, filename)

        df = pd.read_csv(file_path)

        df.columns = df.columns.str.strip()

        df.rename(columns={'published_year': 'Published Year'}, inplace=True)

        df['company_id'] = df['Company'].str.lower().apply(lambda x: match_company_id(x, company_id_lookup_dict))

        df['Indicator'] = df['Indicator'].str.lower().map(lookup_dict)

        df.dropna(subset=['company_id', 'Indicator'], inplace=True)

        quantitative_df = df.copy()

        quantitative_df['quantitative_sentences'] = quantitative_df.apply(lambda row: 1 if pd.notna(row['Value']) and pd.notna(row['Units']) else 0, axis=1)

        quantitative_counts = quantitative_df.groupby(['company_id', 'Company', 'Published Year'])['quantitative_sentences'].sum().reset_index()

        all_quantitative_counts = pd.concat([all_quantitative_counts, quantitative_counts], ignore_index=True)

        indicator_counts = df.groupby(['company_id', 'Company', 'Published Year', 'Indicator']).size().reset_index(name='Count')

        pivot_table = indicator_counts.pivot_table(index=['company_id', 'Company', 'Published Year'], columns='Indicator', values='Count', fill_value=0).reset_index()

        pivot_table = pivot_table[['company_id', 'Company', 'Published Year'] + [col for col in pivot_table.columns if col not in ['company_id', 'Company', 'Published Year']]]

        all_indicator_counts = pd.concat([all_indicator_counts, pivot_table], ignore_index=True)

all_quantitative_counts.to_csv(quantitative_output_file_path, index=False)
all_indicator_counts.to_csv(indicator_output_file_path, index=False)

print("The indicator counts have been saved to:", indicator_output_file_path)
print("The quantitative sentence counts have been saved to:", quantitative_output_file_path)